In [1]:
# https://stackoverflow.com/questions/36798782/scipy-ckdtree-nearest-neighbor-including-zeros-distance
from scipy.spatial import cKDTree
import numpy as np
import pandas as pd

In [2]:
#load
tracks = pd.read_csv('data/tracks.csv', index_col=0, header=[0, 1]);
features= pd.read_csv('data/features.csv', index_col=0, header=[0, 1, 2])
genres=pd.read_csv('data/genres.csv', index_col=0, header=[0])

In [3]:
# Keep only the genre for each track
label=pd.DataFrame({('label','genre','main'): tracks['track']['genre_top'],
                    ('label','genre','sub'): tracks['track']['genres']})

# Keep on the tracks from small subset
label = label[tracks['set','subset'] == 'large']


# Keep only the genres we want
#genre_list = ['Rock','Folk','Hip-Hop']#['Hip-Hop', 'Folk',Rock','Jazz','Classical','Pop','Electronic']
#label = label[label['label']['genre']['main'].isin(genre_list)]

#add sub genres
genres_dict=genres['title'].to_dict()
genres_dict={str(key):genres_dict[key] for key in genres_dict}

#label['label']['genre']['sub']=label['label']['genre']['sub'].str.replace('[^0-9| ]','').str.split(' ').apply(lambda x:'|'.join([genres_dict[k] for k in x ]))

#dfmax=abs(features).max()
#features=features.divide(dfmax)

# Merging
dataSet = features.merge(label, left_index=True, right_on='track_id')
dataSet.head()

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
20           -0.193837 -0.198527  0.201546  0.258556  0.775204  0.084794   
26           -0.699535 -0.684158  0.048825  0.042659 -0.818967 -0.917123   
30           -0.721487 -0.848560  0.890904  0.088620 -0.445513 -1.271170   
46           -0.119708 -0.858814  2.362546  0.106584 -1.315912 -1.220354   
48           -1.054053  0.932339  0.528064 -1.035338 -1.000682 -1.119304   

feature                                                ...       tonnetz  \
statistics                                             ...           std   
number            07        08        09         10    ...            06   
track_id                                               ...                 
20         -0.289294 -0.816410  0.043851  -0.804761    ...      0.021355   
26         -0.901834 -0.066845 -0.291037  -0.861782    ...      0.023846   
30         -1.240190 -1.343765 -0.905600   2.671522    ...      0.025418   
46         -1.668162 -0.516033 -0.536395  11.025053    ...      0.021310   
48          1.166990 -1.002603 -1.094999  -0.725826    ...      0.025203   

feature           zcr                                                    \
statistics   kurtosis       max      mean    median       min      skew   
number             01        01        01        01        01        01   
track_id                                                                  
20          16.669037  0.469727  0.047225  0.040039  0.000977  3.189831   
26          41.645809  0.250488  0.018388  0.015625  0.000000  4.690596   
30           8.166595  0.546875  0.054417  0.036133  0.002441  2.244708   
46          14.731083  0.223633  0.036601  0.032227  0.000000  2.848736   
48          24.550789  0.371582  0.033015  0.022461  0.000000  4.406733   

feature              label             
statistics       std genre             
number            01  main        sub  
track_id                               
20          0.030993   NaN  [76, 103]  
26          0.014598   NaN  [76, 103]  
30          0.052674   NaN  [76, 103]  
46          0.020713   NaN  [76, 103]  
48          0.039016   NaN  [76, 103]  

[5 rows x 520 columns]

----
#### Creating the KD-Tree for the given dataset:

In [4]:
# Convert to np.array
points_ref = np.array([tuple(x) for x in dataSet.drop(columns='label',level=0).values])

# Construct the KD Tree -> https://en.wikipedia.org/wiki/K-d_tree
tree = cKDTree(points_ref)

`tree` is our KD-Tree that represent the known dataset, we want to include our own song in this dataset without too much computation time

----

#### Loading the custom song features and track id infos:

In [5]:
# Loading custom songs features
features_c = pd.read_csv('data/custom_features.csv', index_col=0, header=[0, 1, 2], sep=',', encoding='utf-8')
tid_c = pd.read_csv('data/custom_tid.csv', index_col=0, header=[0], sep=',', encoding='utf-8')

#features_c=features_c.divide(dfmax)
features_c

feature    chroma_cens                                                    \
statistics    kurtosis                                                     
number              01        02        03        04        05        06   
track_id                                                                   
-1            0.353940 -0.600499 -0.728303 -0.552477  1.530635  2.374295   
-2            2.133935  0.470837  0.038861  0.059704 -0.005942 -0.116764   
-3            7.081917  5.105699  0.667526  1.342866  1.413874  0.491948   
-4           -0.120888 -0.246880 -0.432508 -0.649263 -0.171131 -0.136317   

feature                                               ...      tonnetz  \
statistics                                            ...          std   
number            07        08        09        10    ...           04   
track_id                                              ...                
-1          0.500032 -0.078080  0.234859 -0.036622    ...     0.094174   
-2          1.783537  2.542155  0.143271  0.406001    ...     0.045221   
-3          1.398539  2.676670  0.921653  1.344596    ...     0.055939   
-4         -0.466565  1.878964  0.275685  0.062926    ...     0.081266   

feature                               zcr                                \
statistics                       kurtosis       max      mean    median   
number            05        06         01        01        01        01   
track_id                                                                  
-1          0.018682  0.014828   6.895479  0.393066  0.073572  0.066406   
-2          0.009498  0.010430   3.496868  0.416504  0.098366  0.078613   
-3          0.012561  0.011776   5.765573  0.459473  0.085346  0.071289   
-4          0.014513  0.017297  15.129174  0.567383  0.070954  0.065918   

feature                                   
statistics       min      skew       std  
number            01        01        01  
track_id                                  
-1          0.000000  1.999837  0.046837  
-2          0.006348  1.825609  0.068404  
-3          0.000000  2.090979  0.061508  
-4          0.000977  2.375375  0.039651  

[4 rows x 518 columns]

#### Compute the nearest neighbours in the tree:

In [6]:
# Convert to np.array
features_c_t = np.array([tuple(x) for x in features_c.values])

# Find the 10th nearest points with Euclidean norm in the tree
# https://stackoverflow.com/questions/36798782/scipy-ckdtree-nearest-neighbor-including-zeros-distance
dist, idx = tree.query(features_c_t, k=10, p=2)

----
#### Printing the top 10 track for each custom songs:

In [7]:
# Reset tracks df -> easier to handle TODO
track_r = tracks.reset_index()

# For every added song we fetch from the query
for tid, nodes in enumerate(idx):
    
    # Retrieve the custom id and display the name
    id = features_c.iloc[tid].name
    print("Track id:", id)
    print("Track name:", tid_c[tid_c.index == id].values[0][0])

    # From the neighbours nodes list, retrieve their track id, artist and title
    track_list = list(dataSet.iloc[list(nodes)].index.values)
    results = track_r[track_r['track_id'].isin(track_list)]
    
    # Create a DataFrame to summarize
    r_df = pd.DataFrame({'track_id': track_list, \
                         'artist': results['artist']['name'].values, \
                         'title': results['track']['title'].values, \
                         'distance': dist[tid]})
   
    display(r_df)
    print('\n')

Track id: -1
Track name: The Hit - Tupper Ware Remix Party (feat. Ninja Sex Party)


,track_id,artist,title,distance
0,82717,The Poo Touchers,ya ya,609.093504
1,41502,Reason Enough,Time After Time,644.669134
2,83836,Audace,l'amant des femmes à brioche,652.592402
3,83036,Lido Pimienta,Mueve,661.352005
4,23083,Lame Drivers,Memorial Day (Give Applause),661.402475
5,114820,Debmaster,Propylène v2,666.825436
6,65435,C.H.E.R.N.O.B.Y.L.,Все вокруг,673.477910
7,15182,Crazy Al,После шизофрении,673.957087
8,10539,Break The Bans,25,675.338846
9,79248,Pataphysics,You Risk,676.892650




Track id: -2
Track name: 000002


,track_id,artist,title,distance
0,128972,Bullroarer,Bugga Unga,468.941989
1,75141,Chapelle 59,Seconde,553.219282
2,148041,Pompey,Five-time,682.036281
3,91151,+HIRS+,We Love Choco-Pa (Melt Banana),685.075855
4,45318,Ergo Phizmiz,Troika for the Colonel,685.101586
5,120423,Pataphysics,Less Clay / More Pool,708.360261
6,147075,Matt Oakley,Delivery Guy,718.890377
7,114832,Elvis Herod,Welcome To The Past,724.180976
8,66267,Matt Oakley,Augustine's Restless Heart,725.053253
9,48512,Mise,It's Crazy!,732.157800




Track id: -3
Track name: AWOL_-_03_-_Food


,track_id,artist,title,distance
0,19254,Diablo Swing Orchestra,Infralove,486.548207
1,83036,Эксперимент,Реформатор,555.284844
2,124765,Crazy Al,После шизофрении,564.504125
3,111069,The Impossebulls,One Man's Dope,609.532080
4,111568,Jarby Mc Coy,collage 1,609.532080
5,144530,Jarby Mc Coy,collage 1,638.430012
6,110748,Matt Oakley,Pipe Dreamz,663.752983
7,83007,Indikings,Ninja,666.378396
8,117297,Ayla Nereo,Ayla Nereo - Through The Cracks (Queued Time r...,670.798664
9,152535,Yshwa,partisan,682.082252




Track id: -4
Track name: 01 Out of the Black


,track_id,artist,title,distance
0,38157,"Arivedergrüt, Wwölfli !",Drunken Boosterfly,547.077398
1,148044,The Munitionettes,The More You Got,566.420954
2,95704,Limbs Bin,Citadels (Instrumental),599.496489
3,137477,Zreen Toyz,Abstract Nonsense,626.302032
4,95115,Dj Kannibal Freak,The Cannibal Generals Of Liberia,657.831919
5,97121,Zreen Toyz,[cwk0024] Classwar Karaoke (Zreen Toyz) - 86.A...,657.831919
6,43300,Wehwalt & Zreen Toyz,Wehwalt & Zreen Toyz - Méandres de nuit,675.100106
7,68835,ИЗ-ПОД ЗЕМЛИ,Как Ramones,679.269196
8,146596,Mise,Fear,692.140661
9,154652,Regular Gonzales,Lost Cause,699.468862
